In [47]:
"""
Build database of the CSV files with the repective table names.
"""
import csv, sqlite3

con = sqlite3.connect("SANFRAN.db")
con.text_factory = str
cur = con.cursor()

# create nodes table
cur.execute("CREATE TABLE nodes (id, lat, lon, user, uid, version, changeset, timestamp);")
with open('nodes.csv','rb') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'], i['lat'], i['lon'], i['user'], i['uid'], i['version'], i['changeset'], i['timestamp']) \
             for i in dr]

cur.executemany("INSERT INTO nodes (id, lat, lon, user, uid, version, changeset, timestamp) \
                VALUES (?, ?, ?, ?, ?, ?, ?, ?);", to_db)
con.commit()

#create nodes_tags table
cur.execute("CREATE TABLE nodes_tags (id, key, value, type);")
with open('nodes_tags.csv','rb') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'], i['key'], i['value'], i['type']) for i in dr]

cur.executemany("INSERT INTO nodes_tags (id, key, value, type) VALUES (?, ?, ?, ?);", to_db)
con.commit()

#Create ways table
cur.execute("CREATE TABLE ways (id, user, uid, version, changeset, timestamp);")
with open('ways.csv','rb') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'], i['user'], i['uid'], i['version'], i['changeset'], i['timestamp']) for i in dr]

cur.executemany("INSERT INTO ways (id, user, uid, version, changeset, timestamp) VALUES (?, ?, ?, ?, ?, ?);", to_db)
con.commit()

#Create ways_nodes table
cur.execute("CREATE TABLE ways_nodes (id, node_id, position);")
with open('ways_nodes.csv','rb') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'], i['node_id'], i['position']) for i in dr]

cur.executemany("INSERT INTO ways_nodes (id, node_id, position) VALUES (?, ?, ?);", to_db)
con.commit()

#Create ways_tags table
cur.execute("CREATE TABLE ways_tags (id, key, value, type);")
with open('ways_tags.csv','rb') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'], i['key'], i['value'], i['type']) for i in dr]

cur.executemany("INSERT INTO ways_tags (id, key, value, type) VALUES (?, ?, ?, ?);", to_db)
con.commit()

In [46]:
con = sqlite3.connect("SANFRAN.db")
cur = con.cursor()
cur.execute('DROP TABLE IF EXISTS nodes')
con.commit()
cur.execute('DROP TABLE IF EXISTS nodes_tags')
con.commit()
cur.execute('DROP TABLE IF EXISTS ways')
con.commit()
cur.execute('DROP TABLE IF EXISTS ways_tags')
con.commit()
cur.execute('DROP TABLE IF EXISTS ways_nodes')
con.commit()

In [49]:
def number_of_nodes():
    result = cur.execute('SELECT COUNT(*) FROM nodes')
    return result.fetchone()[0]
number_of_nodes()

6278429

In [51]:
def number_of_ways():
    result = cur.execute('SELECT COUNT(*) FROM ways')
    return result.fetchone()[0]

770438

In [53]:
def number_of_unique_users():
    result = cur.execute('SELECT COUNT(DISTINCT(uid)) FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways)')
    return result.fetchone()[0]

2666

In [67]:
def top_users():
    users = []
    for row in (cur.execute('SELECT user, COUNT(*) as num FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) GROUP BY user ORDER BY num DESC Limit 10')):
        users.append(row)
    return users
top_users()

[('andygol', 1288108),
 ('ediyes', 912011),
 ('Luis36995', 703573),
 ('dannykath', 445309),
 ('RichRico', 404022),
 ('Rub21', 393086),
 ('calfarome', 185572),
 ('oldtopos', 167231),
 ('KindredCoda', 149861),
 ('karitotp', 134929)]

In [71]:
def top_amenities():
    amenities = []
    for row in (cur.execute('SELECT value, COUNT(*) as num FROM nodes_tags WHERE key = "amenity" GROUP BY VALUE ORDER BY num DESC LIMIT 10')):
        amenities.append(row)
    return amenities
top_amenities()

[('restaurant', 2884),
 ('bench', 1162),
 ('cafe', 970),
 ('place_of_worship', 702),
 ('post_box', 684),
 ('school', 590),
 ('fast_food', 580),
 ('bicycle_parking', 558),
 ('drinking_water', 507),
 ('toilets', 401)]

In [107]:
def popular_fast_foods():
	for row in cur.execute('SELECT nodes_tags.value, COUNT(*) as num \
            FROM nodes_tags \
                JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value="fast_food") i \
                ON nodes_tags.id=i.id \
            WHERE nodes_tags.key="name" \
            GROUP BY nodes_tags.value \
            ORDER BY num DESC Limit 10'):
            print row;
popular_fast_foods()

('Subway', 63)
("McDonald's", 26)
('Taco Bell', 22)
('Burger King', 21)
('Jamba Juice', 18)
('Chipotle', 11)
('Jack in the Box', 9)
('KFC', 8)
("Togo's", 7)
("Wendy's", 6)


In [110]:
def popular_cafes():
	for row in cur.execute('SELECT nodes_tags.value, COUNT(*) as num \
            FROM nodes_tags \
                JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value="cafe") i \
                ON nodes_tags.id=i.id \
            WHERE nodes_tags.key="name" \
            GROUP BY nodes_tags.value \
            ORDER BY num DESC Limit 10'):
            print row;
popular_cafes()

('Starbucks', 126)
("Peet's Coffee & Tea", 24)
('Starbucks Coffee', 18)
("Peet's Coffee", 10)
('Philz Coffee', 10)
("Peet's Coffee and Tea", 8)
('Quickly', 7)
('Blue Bottle Coffee', 5)
('Jamba Juice', 5)
('Royal Ground Coffee', 5)
